<H1>Relationship between IL and price change in a stableswap pool</H1>
Here we investigate the impermanent loss experienced by an LP in a stableswap pool when one of the assets depegs (its value deviates from 1USD). Because the assets in a stableswap pool are expected to remain stable, the pool is optimized to give a better execution price, with less slippage, at the expense of being more vulnerable to impermanent loss in the event of a price change. For the sake of simplicity, the LP both deposits and withdraws a proportional amount of each token (so that these trades don't induce further price changes). Note that the general dynamics and scale of the impermanent loss will hold regardless of which asset the LP chooses to deposit or withdraw.
<br><br>
An important parameter of the stableswap pool is its amplification, which affect the shape of the price curve. At an amplification of 0, the stableswap pool behaves identically to an XYK pool, whereas at an amplification of infinity, the price of each asset remains at exactly 1 until the pool runs out of liquidity. At an intermediate value, the price remains stable at a wider range of balances than it would in an XYK pool, but hits a 'cliff' when the balance shifts too far in either direction. Higher values give a more stable price but a steeper cliff, and are more appropriate when there is high confidence in the stability of the underlying assets. Here we will graph the IL dynamics for a range of amplification values.

In [1]:
import sys
import os
import glob
import random
import copy
from IPython.display import display, Markdown
sys.path.append('../..')

from model import processing
from matplotlib import pyplot as plt

from model import run
from model import plot_utils as pu
from model.amm.omnipool_amm import OmnipoolState, cash_out_omnipool, value_assets, usd_price, lrna_price
from model.amm.stableswap_amm import StableSwapPoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import omnipool_arbitrage, invest_all, price_sensitive_trading
from model.amm.global_state import GlobalState, fluctuate_prices, historical_prices

# same seed, same parameters = same simulation result
random.seed(42)
# price_list = processing.import_binance_prices(['BTC', 'ETH', 'DOT'], start_date='Jan 1 2023', days = 120)

assets = {
    'USDA': {'usd price': 1, 'weight': 0.25},
    'USDB': {'usd price': 1, 'weight': 0.25},
    'USDC': {'usd price': 1, 'weight': 0.25},
    'USDD': {'usd price': 1, 'weight': 0.25},
}

assert sum([t['weight'] for t in assets.values()]) == 1

initial_tvl = 1000000

initial_state = StableSwapPoolState(
    tokens={
        tkn: initial_tvl * assets[tkn]['weight'] for tkn in assets
    },
    unique_id='stableswap',
    amplification=100,
    trade_fee=0.003
)

initial_agent = Agent(
    holdings={'USDA': 5000}
)


In [9]:
print(f"Agent adds liquidity: {initial_agent.holdings['USDA']} USDA. Trade fee = {initial_state.trade_fee}")
test_agent = initial_agent.copy()
test_pool = initial_state.copy().add_liquidity(
    agent=test_agent,
    tkn_add='USDA',
    quantity=initial_agent.holdings['USDA']
)
no_fee_agent = initial_agent.copy()
no_fee_state = initial_state.copy()
no_fee_state.trade_fee = 0
no_fee_state.add_liquidity(
    agent=no_fee_agent,
    tkn_add='USDA',
    quantity=initial_agent.holdings['USDA']
)
print(f"Agent receives: {test_agent.holdings['stableswap']} stableswap shares. Effective fee = {(1 - test_agent.holdings['stableswap'] / no_fee_agent.holdings['stableswap']) * 100}%")

Agent adds liquidity: 5000 USDA. Trade fee = 0.003
Agent receives: 4996.244215616607 stableswap shares. Effective fee = 0.07499994312618607%
